In [ ]:
# Chapter 2 regex text normalization edit distance

In [15]:
from IPython.display import Image
from IPython.display import HTML

In [16]:
Image('image url here')
HTML('web or frame url here')



lemmatization: determining if words have the same root, ex, sung, sang, sings - all root is sing


In [17]:
# regex module
import re

# Regex basics

In [ ]:
re.match(r'\\ko*', r'\ko')

In [ ]:
p = re.compile('[a-z]+')

In [ ]:
p 

In [ ]:
p.match('')

In [ ]:
m = p.match('tempo')

In [ ]:
m

1. group() 	Return the string matched by the RE
2. start() 	Return the starting position of the match
3. end() 	Return the ending position of the match
4. span() 	Return a tuple containing the (start, end) positions of the match

1. match() 	Determine if the RE matches at the beginning of the string.
2. search() 	Scan through a string, looking for any location where this RE matches.
3. findall() 	Find all substrings where the RE matches, and returns them as a list.
4. finditer() 	Find all substrings where the RE matches, and returns them as an iterator.

In [ ]:
#ex
m.group(0)

In [ ]:
m.start()

In [ ]:
m.span()

In [ ]:
print(p.match(':::message'))

In [ ]:
print(p.search('::::message')); 

In [ ]:
m = p.search('::::message'); m

In [ ]:
m.group()

In [ ]:
re.search('koo', 'kootkikkoo')

In [ ]:
re.search('[Kk1]', 'kooty')


1. [^A-z] - not an upper case letter
2. [^\.] - not a period ^ - means not now, but only at the beginnning
3. [a^] - string a^
3. ? - optional element


In [ ]:
re.search('cats?', 'cat') # zero or none previous char

In [ ]:
re.search('cats?', 'cat')

1. *(Kleeny star) - zero or more of previous


In [ ]:
re.search('ba*', 'baa')

In [ ]:
re.match('ba*', 'cb')# not match

In [ ]:
# . any single char
re.match('a.d', 'aad')

In [ ]:
re.match('a.d', 'ad') # not match

In [ ]:
#  $ - matches the end of line _$

In [ ]:
re.search('sd$', 'sd')

In [ ]:
re.search(' $', ' ')

In [ ]:
re.search('s\\\\', r's\\')

In [ ]:
# \b word boundary
re.search(r'\b99\b', r's 99 ')

In [ ]:
re.search(r'\b99\b', r's $99 ') # match because $ is not digot underscore, digits or letter

In [ ]:
# | disjunction matches left or right

In [ ]:
re.search(r'cat|dog', r' is enemy of  cat ')

In [91]:
re.search(r'[\w](b)', r' guppb') # round brackets (,) takes precedence 

<_sre.SRE_Match object; span=(4, 6), match='pb'>

In [ ]:
# also used to:
re.search('( Column [0-9]+ *)*', ' Column 9    and')

In [9]:
# simple match word the

p = re.search('(^|[^a-zA-Z])[tT]he([^a-zA-Z]|$)', ' the rock')# matches the at he beginning or end
                                                             # or not surrounded by letters


NameError: name 're' is not defined

In [ ]:
p;p is None


In [ ]:
p

In [ ]:
re.search('\\b[0-9]+(\.[0-9]+)? *(GB|[Gg]igabytes?)\\b', '$345.00 GB 300') # matches price in dol and hd spec

In [ ]:
# {<number>} -  exactly number occurences of prievous expr

In [ ]:
re.search('(kocham koty){2}', 'kocham kotykocham koty') # mtch kocham kot exactly twice

# Regex substitution

In [10]:
re.match('the (.*)er they were, the \1er they will be', 'the bigger they were, the bigger they will be')

NameError: name 're' is not defined

In [11]:
re.sub(r'....', r'dranie', r'supa')

NameError: name 're' is not defined

In [ ]:
! sed -e 's/a/b/g' text.txt

In [ ]:
! cat text.txt

In [ ]:
# Lemma is set of lexical forms which have the same steam and the same word sense cat, cats
# types: number of differentt words
# tokens: number of words

# text normalization


In [ ]:
! tr -sc 'a-zA-Z' '\n' < text.txt | head # split using new line as a separator


In [ ]:
! tr -sc 'a-zA-Z' '\n' < text.txt | tr A-Z a-z | sort | uniq -c | head #  collapse all to lower case, sort and collapse 

In [ ]:
! tr -sc 'a-zA-Z' '\n' < text.txt | tr A-Z a-z | sort | uniq -c | sort -n -r # and finally sort numerically to get freq


Tokenization - separating text into tokens /words - problems with prices, shortcuts, dates, numbers, mail addresses, html addresses

In [1]:
# max match algorithm
D = ['danny', 'kondory', 'a', 'the','to','has', 'been', 'unable', 'go' ]
#D = ['d', 'a', 'n' ]
def max_match(sentence, dictionary):
    if not sentence:
        return ""
    for i in range(len(sentence), -1, -1):
        first_word = sentence[:i]
        remainder = sentence[i:]
        if first_word in dictionary:
            return first_word + " " +  max_match(remainder, dictionary)
    first_word = sentence[0]
    remainder = sentence[1:]
    return first_word + max_match(remainder, dictionary)
    

In [5]:
sent = "the" # len = 3

sentence = sent.split()
print(max_match("thebeenkondorygounable", D))
#sent[3:]
#type(sent[3:])

the been kondory go unable 


In [ ]:
st1 = [1, 3 ,4 ,5 ]
st1[1:].append(st1[0])
print(st1)

# Edit distance algorithm

In [ ]:
# edit distance in both recursive (not to be used to slow) and dynamic programming version
# recursive:
def d(s1, s2):
    if len(s1) == 0 and len(s2) == 0:
        return 0
    elif len(s1) == 0 and len(s2) != 0:
        return len(s2)
    elif len(s2) == 0 and len(s1) != 0:
        return len(s1)
    else:
        return min(d(s1[:-1], s2[:-1]) + comp_lasts(s1[-1], s2[-1]),
                   d(s1, s2[:-1]) + 1,
                   d(s1[:-1], s2) + 1)

In [256]:
# dynaic programming:
def editDistDP(str1, str2, m, n):
    # Create a table to store results of subproblems
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]

    # Fill d[][] in bottom up manner
    for i in range(m + 1):
        for j in range(n + 1):

            # If first string is empty, only option is to
            # isnert all characters of second string
            if i == 0:
                dp[i][j] = j  # Min. operations = j

            # If second string is empty, only option is to
            # remove all characters of second string
            elif j == 0:
                dp[i][j] = i  # Min. operations = i

            # If last characters are same, ignore last char
            # and recur for remaining string
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]

            # If last character are different, consider all
            # possibilities and find minimum
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],  # Insert
                                   dp[i - 1][j],  # Remove
                                   dp[i - 1][j - 1])  # Replace

    return dp[m][n]

In [ ]:
st1 = "sunday euheu jodjod"
st2 = "saturdaydkwdwmmdwod"
print(editDistDP(st1, st2, len(st1), len(st2)))

In [92]:
# find set of all alphabetic strings
re.search(r'(^|[ ])(a-z(b))+([ ]|$)', r'werb')  

In [342]:
re.search(r"^[-+]?[0-9]+\.[0-9]+$", r"-22.9") # matches float poiunt numbers only in form +-2.2

In [343]:
re.search(r'\b[Zz]osia\b', r'zosf ia')

<_sre.SRE_Match object; span=(0, 5), match='zosia'>

In [410]:
re.search(r"^(https?:\/\/|www\.)?([a-z]+\.)+[a-z]+((\/[a-z]+)*[a-z]+\.[a-z]+)*$", r"www.hdomain.rozrywka.onet.pl/xyx/abc/uff.html")

<_sre.SRE_Match object; span=(0, 45), match='www.hdomain.rozrywka.onet.pl/xyx/abc/uff.html'>

In [242]:
def uns_num(x):
    if x < 0:
        return (x + 2**32)
    else:
        return x
    
def bound(a, x, b):
    
    return uns_num(x - a) <= b - a
   



In [432]:
print(bound(1, -10, 10))

False


In [434]:
re.search(r"(?<=title=\").*?(?=\")", r"title=\"Image\"")

In [7]:
#re.search(r"^dim( \. )?(\\)?t[0-9]+$", r"dim\t65")

In [19]:
shell = 1
entry = {}                                                                                         
entry['title'] = 'Dive into history, 2009 edition'
entry['article_link'] = 'http://diveintomark.org/archives/2009/03/27/dive-into-history-2009-edition'
entry['comments_link'] = None
entry['internal_id'] = b'\xDE\xD5\xB4\xF8'
entry['tags'] = ('diveintopython', 'docbook', 'html')
entry['published'] = True
import time
entry['published_date'] = time.strptime('Fri Mar 27 22:20:42 2009')                                
entry['published_date']

time.struct_time(tm_year=2009, tm_mon=3, tm_mday=27, tm_hour=22, tm_min=20, tm_sec=42, tm_wday=4, tm_yday=86, tm_isdst=-1)

In [20]:
l = [1,entry]
import pickle
with open('entry.pickle', 'wb') as f:
    pickle.dump(l[1], f)
#del(entry)

#del(entry)

In [135]:
with open('entry.pickle', 'rb') as f:
    l[1] = pickle.load(f)
l[1]
#entry['00added4'] = 'test4 data'
#del(entry)

{'article_link': 'http://diveintomark.org/archives/2009/03/27/dive-into-history-2009-edition',
 'comments_link': None,
 'internal_id': b'\xde\xd5\xb4\xf8',
 'published': True,
 'published_date': time.struct_time(tm_year=2009, tm_mon=3, tm_mday=27, tm_hour=22, tm_min=20, tm_sec=42, tm_wday=4, tm_yday=86, tm_isdst=-1),
 'tags': ('diveintopython', 'docbook', 'html'),
 'title': 'Dive into history, 2009 edition'}

In [136]:
prices = [5, 12, 45]

In [142]:
for i in range(0, len(prices)):
    exec("price%d = %s" % (i + 1, repr(prices[i])));
type(price1)

int

In [80]:
import pickle
doc1 = """I did enact Julius Caesar  I was killed
i the Capitol , Brutus killed me"""
doc2 = """So let it be with Caesar. The noble Brutus
hath told you Caesar was ambitious:"""
cnt_docs = 2
i = 0
'''while i < cnt_docs:
    s = "doc%s.pickle" % i
    with open(s, 'wb') as f:
         pickle.dump(s, f)
         print(s, type(s))
       
    i += 1
## we have d0.pickle and d1.pickle on disk   
'''

'while i < cnt_docs:\n    s = "doc%s.pickle" % i\n    with open(s, \'wb\') as f:\n         pickle.dump(s, f)\n         print(s, type(s))\n       \n    i += 1\n## we have d0.pickle and d1.pickle on disk   \n'

In [70]:
i = 0
while i < cnt_docs:
    s = "doc%s.pickle" % i
    with open(s, 'rb') as f:
        #print(s)
        pickle.load(f)
        print(f.readline())
    i += 1
f1 = open('doc0.pickle', 'rb')
print(f1.readlines())
f1.close()

b''
b''
[b'\x80\x03X\x0b\x00\x00\x00doc0.pickleq\x00.']


In [69]:
! cat doc0.pickle

�X   doc0.pickleq .

In [3]:
def find(elem, d_list):
    l_return = []
    i = 0
    for x in d_list:
        if elem in x:
            l_return.append(i)
        i += 1
    return (l_return, len(l_return))

In [16]:
set(doc1.lower().split())

{',',
 'brutus',
 'caesar',
 'capitol',
 'did',
 'enact',
 'i',
 'julius',
 'killed',
 'me',
 'the',
 'was'}

# Inverse index

In [82]:
# This works assuming, that a two single documents will fit into memory...
from collections import deque

d1 = ['i', 'did', 'enact', 'julius', 'caesar', 'i', 'was', 'kill' , 'i', 'the', 'capitol',  'brutus','me']
d1 = set(d1)
d2 = ['i', 'so', 'let', 'it', 'be', 'with' , 'caesar', 'the', 'noble', 'brutus', 'hath', 'told', 'you', 
      'caesar', 'was', 'ambitious']
d2 = set(d2)
files = {'1': d1, '2': d2}
# d1, d2 lists with serialized, tokenized documents need to be procesed one by one in a loop,
# counted and dumped to disk, we already have a dict with files and pointers to them 
doc_list = deque()
doc_list.append(d1)
doc_list.append(d2)
##print(doc_list)

tokens = d1.union(d2)
## print("tokens", tokens)
inv_index = {}
for x in tokens:
    tmp, tmp1 = find(x, doc_list)
    inv_index[(x, tmp1)] = tmp 

def AND(w1, w2):
    tmp0 =  set(inv_index[find_key(w1, inv_index)])
    tmp1 =  set(inv_index[find_key(w2, inv_index)])
    indexes = tmp0.intersection(tmp1)
    ret_list = []
    for x in indexes:
        ret_list.append(x)
    return ret_list

def OR(w1, w2):
    tmp0 =  set(inv_index[find_key(w1, inv_index)])
    tmp1 =  set(inv_index[find_key(w2, inv_index)])
    indexes = tmp0.union(tmp1)
    ret_list = []
    for x in indexes:
        ret_list.append(x)
    return ret_list
    
def NOT(w1):
    tmp0 =  set(inv_index[find_key(w1, inv_index)])
    indexes = list(tmp0)
    ret_list = []
    for x in range(len(doc_list)):
        if x not in indexes:
            ret_list.append(x)
    return ret_list

def NOT1(l1):
    #tmp0 =  set(inv_index[find_key(w1, inv_index)])
    indexes = l1
    ret_list = []
    for x in range(len(doc_list)):
        if x not in indexes:
            ret_list.append(x)
    return ret_list
        
    
    
AND('the', 'i')
OR('let', 'you')



s = set()
s.add("A")
s.add("B")
s.add("C")
not "C" in s
inv_index[find_key('the', inv_index)]

(inv_index[find_key('it', inv_index)] or inv_index[find_key('let', inv_index)]) and( not inv_index[find_key('capitol', inv_index)])

list(inv_index.values())
list(set(inv_index[find_key('i', inv_index)]))
#OR('let', 'you')
OR('so', 'i')
#NOT('did')
#AND('did', 'it')
#NOT1([0])


'doc1'

In [5]:
def find(elem, d_list):
    l_return = []
    i = 0
    for x in d_list:
        if elem in x:
            l_return.append(i)
        i += 1
    return (l_return, len(l_return))

def find_key(x, d):
    for y in d.keys():
        if x in y:
            return y
    return "word not in a texts"

In [186]:
! mv daily/language_processing.ipynb daily/26-01-2017_language_processing.ipynb

In [48]:
l = [0, 1, 2]
from collections import deque

In [231]:
d = deque()
d.appendleft(1)
d
d.appendleft(2)
d


2

In [49]:
import pickle
with open('l[2].pickle', 'wb') as f:
    pickle.dump(l[2], f)


In [50]:
with open('l[2].pickle', 'rb') as f:
    l.append(pickle.load(f))
l[2]

#entry['00added4'] = 'test4 data'
#del(entry)

2

In [238]:
l = [0, 1, 2]

In [248]:
l

[0, 1]

In [42]:
f = open('text.txt', 'r')
print(f.readlines())
f.close()


['hdiwhd\n', 'dwihdw\n', 'niwd']


In [39]:
%%writefile text.txt
hdiwhd
dwihdw
niwd


Writing text.txt


In [287]:
%mv /home/lion/projects/daily_uploads/language_processing.ipynb /home/lion/projects/daily_uploads/26-01-2017_inverted_index_notes.ipynb

In [27]:
import sqlite3
! ls /home/lion/projects/python/nlp/

american_fkah.ipynb  functions_and_tree_class.py  regexp.py
Awk		     main_function.py		  tests
chap1		     parse_tree.py		  texts
Crawler.ipynb	     porpositional_logic_eval.py
daily_uploads	     __pycache__


In [31]:
%cp /home/lion/projects/python/nlp/porpositional_logic_eval.py /home/lion/projects/daily_uploads/

In [33]:
from porpositional_logic_eval import *

In [34]:
%cat porpositional_logic_eval.py

import string
import re
import operator as op
import functools
from itertools import product


class BinaryTree: 
    def __init__(self, rootObj):
        self.key = rootObj
        self.parent = None
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            t = BinaryTree(newNode)
            self.leftChild = t
            t.parent = self
        else:
            t = BinaryTree(newNode)
            t.parent = self
            t.leftChild = self.leftChild
            self.leftChild.parent = t
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            t = BinaryTree(newNode)
            self.rightChild = t
            t.parent = self
        else:
            t = BinaryTree(newNode)
            t.parent = self
            t.rightChild = self.rightChild
            self.rightChild.parent = t
            self.rightChild = t

    def getRightChild(

In [39]:
F = "~(~1 || 0)"
tr = build_parse_tree(F)
inorder_traversal(tr)
evaluate_parse_tree(tr)

True
~
||
False
~


True

In [ ]:
def query_tree_evaluate(tree):
    opers = {'||': OR, '&&': AND, '~': NOT: implication, '==': equality, 'XR': xor} 
    leftT = tree.getLeftChild()
    rightT = tree.getRightChild()
    if leftT and not rightT:
        fn = opers[tree.getRootVal()]
        return fn(evaluate_parse_tree(leftT))
    elif leftT and rightT:
        fn = opers[tree.getRootVal()]
        return fn(evaluate_parse_tree(leftT), evaluate_parse_tree(rightT))
    else:
        return tree.getRootVal()